In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
import matplotlib.pyplot as plt
import json

%matplotlib inline

# 한글 폰트 문제 해결
import platform
from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')


# 노란색 글씨의 warining 문구 강제 생략
import warnings
warnings.filterwarnings(action='ignore')
    

# 차트 축 <- 음수 부호 지원
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False


print('numpy version - ' , np.__version__)
print('pandas version - ' , pd.__version__)

# 크롤링
from bs4 import BeautifulSoup
from urllib.request import urlopen , urlretrieve
from urllib.error   import HTTPError
from urllib.error   import URLError

import requests
import re


numpy version -  1.21.5
pandas version -  1.4.2


In [ ]:
# HTTP GET Request
req = requests.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx?sort=HRA_RT')

# HTML 소스 가져오기
html = req.text

# BeautifulSoup으로 html소스를 python객체로 변환하기
soup2 = BeautifulSoup(html, 'html.parser')

# html에서 원하는 부분 가져오기 : 오류나는 부분! 
soup2.select('.record_result')

In [2]:
try :
    html = urlopen('https://pythondojang.bitbucket.io/weather/observation/currentweather.html')
except HTTPError as he :
    print('http error')
except URLError  as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read() , 'html.parser')

In [10]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>도시별 현재날씨 &gt; 지상관측자료 &gt; 관측자료 &gt; 날씨 &gt; 기상청 </title>
<link href="http://www.kma.go.kr/favicon2.ico" rel="shortcut icon"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="/share/css/import.css?20160530" rel="stylesheet" type="text/css"/>
<script src="/share/js/jquery-1.7.1.min.js" type="text/javascript"></script>
<script src="/share/js/common.js?ver=20150417" type="text/javascript"></script>
<link href="/share/css/warninglayer.css" rel="stylesheet" type="text/css"/>
<!--[if gte IE 7]><link rel="stylesheet" type="text/css" href="/share/css/ie7.css" /><![endif]-->
<meta content="기상청 " name="title"/>
<meta content="기상청" name="author"/>
<meta content="날씨, 기상청" name="keywords"/>
<meta content="서울시 기상청 사이트입니다." name="description"/>
<meta content="IE=Emul

In [13]:
#table = soup.select('.table_develop3')
table = soup.find('table' , {'class' : 'table_develop3'})
print('type - ' , type(table))
print()
print(table)

type -  <class 'bs4.element.Tag'>

<table class="table_develop3" summary="기상실황표로 지점, 날씨, 기온, 강수, 바람, 기압등을 안내한 표입니다.">
<caption>기상실황표</caption>
<colgroup>
<col width="14%"/>
<col width="12%"/>
<col width="7%"/>
<col width="5%"/>
<col width="8%"/>
<col width="5%"/>
<col width="6%"/>
<col width="5%"/>
<col width="8%"/>
<col width="5%"/>
<col width="8%"/>
<col width="6%"/>
<col width="*%"/>
</colgroup>
<thead>
<tr class="table_header" id="table_header1">
<th class="top_line" rowspan="2" scope="col">
		지점
		</th>
<th class="top_line" colspan="4" scope="col">날씨</th>
<th class="top_line" colspan="3" scope="col">기온(℃)</th>
<th class="top_line" colspan="2" scope="col">강수</th>
<th class="top_line" colspan="2" scope="col">바람</th>
<th class="top_line" scope="col">기압(hPa)</th>
</tr>
<tr class="table_header" id="table_header2">
<th class="nm" scope="col">현재일기 </th>
<th class="nm" scope="col">시정<br/>km</th>
<th class="nm" scope="col">운량<br/>1/10</th>
<th class="nm" scope="col">중하운량</th>
<th class="nm

In [ ]:
# 지점 현재 온도 습도 데이터프레임으로 생성

In [48]:
location_lst = []
temp_lst = []
humditiy_lst = []

for tr in table.find_all('tr') :
    tds = tr.find_all('td')
    for td in tds :
        if td.find('a') :
            location_lst.append(td.find('a').get_text())
            temp_lst.append(tds[5].get_text())
            humditiy_lst.append(tds[9].get_text())
            

temp_frm = pd.DataFrame({ 'location' : location_lst ,
                    'temp' : temp_lst ,
                    'humidity' : humditiy_lst})
temp_frm
                   
            


,location,temp,humidity
0,서울,25.6,30
1,백령도,18.4,62
2,인천,20.8,54
3,수원,25.0,41
4,동두천,24.9,34
...,...,...,...
90,합천,25.1,32
91,밀양,24.7,34
92,산청,24.8,41
93,거제,23.1,57


In [49]:
# https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx
# https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx?sort=HRA_RT
# 선수명 , avg , h , hr , rbi 
# avg 상위 5명의 선수만 시각화

In [121]:
try :
    html = urlopen('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx')
except HTTPError as he :
    print('http error')
except URLError  as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read() , 'html.parser')

In [122]:
soup


<!DOCTYPE html>

<html lang="ko">
<head><title>
	기록실 | 기록 | KBO
</title><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="text/javascript" http-equiv="Content-Script-Type"/><meta content="text/css" http-equiv="Content-Style-Type"/><meta content="IE=edge, Chrome" http-equiv="X-UA-Compatible"/><meta content="KBO" name="title"/><meta content="스포츠투아이" name="author"/><meta content="KBO, 한국야구위원회, 프로야구, KBO 리그, 퓨처스리그, 프로야구순위, 프로야구 일정" name="keywords"/><meta content="width=1200" name="viewport"/>
<!-- OG -->
<meta content="website" property="og:type"/><meta content="KBO 홈페이지" property="og:title"/><meta content="KBO, 한국야구위원회, 프로야구, KBO 리그, 퓨처스리그, 프로야구순위, 프로야구 일정" property="og:description"/><meta content="https://lgcxydabfbch3774324.cdn.ntruss.com/KBO_IMAGE/KBOHome/resources/images/common/open_graph_kbo_2022.png" property="og:image"/><meta content="https://www.koreabaseball.com/" property="og:url"/>
<link href="//lgcxydabfbch3774324.cdn.ntruss.com/KBO_IMAGE/KBOH

In [728]:
# table = soup.select('.table_develop3')
# table = soup.find('table' , {'class' : 'tData01 tt'})

table = soup.find('table' , {'class' : 'tData01 tt'})
table = soup.select('.record_result')
print('type - ' , type(table))
print()
print(table)

type -  <class 'bs4.element.ResultSet'>

[<div class="record_result">
<table cellpadding="0" cellspacing="0" class="tData01 tt" summary="선수 주루기록으로 순위,선수명,팀명,경기,도루시도,도루성공,도루실패,도루성공률,주루사,견제사를 표시합니다">
<colgroup>
<col width="80"/>
<col width="120"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
</colgroup>
<thead>
<tr>
<th>순위</th>
<th>선수명</th>
<th>팀명</th>
<th><a href="javascript:sort('GAME_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="경기">G</a></th>
<th><a href="javascript:sort('SBA_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루시도">SBA</a></th>
<th><a href="javascript:sort('SB_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루허용">SB</a></th>
<th><a href="javascript:sort('CS_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루실패">CS</a></th>
<th><a href="javascript:sort('SB_RT');" onmouseenter="tooltip(this)" onmouseout

In [107]:

for tr in table.find_all('tr') :
    tds = tr.find_all('td')
    for td in tds :
        print(td)      

In [116]:
import requests
from bs4 import BeautifulSoup

# HTTP GET Request
req = requests.get('https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx?sort=HRA_RT')

# HTML 소스 가져오기
html = req.text

# BeautifulSoup으로 html소스를 python객체로 변환하기
soup2 = BeautifulSoup(html, 'html.parser')

# html에서 원하는 부분 가져오기 : 오류나는 부분! 
soup2.select('.record_result')
html

'\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head><title>\r\n\t기록실 | 기록 | KBO\r\n</title><meta http-equiv="Content-Type" content="text/html; charset=utf-8" /><meta http-equiv="Content-Script-Type" content="text/javascript" /><meta http-equiv="Content-Style-Type" content="text/css" /><meta http-equiv="X-UA-Compatible" content="IE=edge, Chrome" /><meta name="title" content="KBO" /><meta name="author" content="스포츠투아이" /><meta name="keywords" content="KBO, 한국야구위원회, 프로야구, KBO 리그, 퓨처스리그, 프로야구순위, 프로야구 일정" /><meta name="viewport" content="width=1200" />\r\n    <!-- OG -->\r\n    <meta property="og:type" content="website" /><meta property="og:title" content="KBO 홈페이지" /><meta property="og:description" content="KBO, 한국야구위원회, 프로야구, KBO 리그, 퓨처스리그, 프로야구순위, 프로야구 일정" /><meta property="og:image" content="https://lgcxydabfbch3774324.cdn.ntruss.com/KBO_IMAGE/KBOHome/resources/images/common/open_graph_kbo_2022.png" /><meta property="og:url" content="https://www.koreabaseball.com/" />\r\n\r\n\t<link r

In [117]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

url_base = 'https://www.koreabaseball.com/Record/Player/{category}'
category_list = ['HitterBasic/Basic1.aspx', 'PitcherBasic/Basic1.aspx', 
                 'Defense/Basic.aspx', 'Runner/Basic.aspx']

data_list = []

for category in category_list:
    url = url_base.format(category = category)
    req = requests.get(url)

    html = req.text
    soup = bs(html, 'html.parser')
    
    temp_table = soup.find('div', {'class':'record_result'})
    #### get table info
    
    col_tag = temp_table.find_all('td')
    col_list = ['Rank', 'Name', 'Team']
    for col in col_tag:
        try:
            temp_value = col.attrs['data-id']
            if temp_value not in col_list:
                col_list.append(temp_value)
            else:
                break
        except:
            pass
        
    temp_data = pd.DataFrame(columns = col_list)

        
    i = 0
    index = 0
    col_len = len(col_list)
    while True:
        try:
            temp_data.loc[i] = [x.text for x in temp_table.find_all('td')[index:index + col_len]]
            i += 1
            index += col_len
        except:
            break
    
    data_list.append(temp_data)

In [118]:
data_list

[Empty DataFrame
 Columns: [Rank, Name, Team]
 Index: [],
 Empty DataFrame
 Columns: [Rank, Name, Team]
 Index: [],
    Rank   Name Team POS_SC GAME_CN START_GAME_CN DEFEN_INN2_CN ERR_CN POFF_CN  \
 0     1    배정대   KT    중견수     144           134      1195 2/3      3       0   
 1     1    박해민   LG    중견수     144           142          1205      2       0   
 2     3    오지환   LG    유격수     142           137          1167     16       0   
 3     4    황재균   KT    3루수     138           133      1147 2/3     12       0   
 4     4    송성문   키움    3루수     138           137          1216     15       0   
 5     4    박성한  SSG    유격수     138           131          1176     24       0   
 6     7    이지영   키움     포수     137           102       994 2/3     11       0   
 7     7    김선빈  KIA    2루수     137           134      1134 1/3     10       0   
 8     9    유강남   LG     포수     134           115      1008 1/3      7       1   
 9     9    이정후   키움    중견수     134           132      1155 1/3 

In [ ]:
# https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&date=20221127

In [145]:
try :
    html = urlopen('https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&date=20221127')
except HTTPError as he :
    print('http error')
except URLError  as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read() , 'html.parser')

In [147]:
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="me2:image">
<meta content="네이버영화 " property="me2:post_tag">
<meta content="네이버영화" property="me2:category1"/>
<meta content="" property="me2:category2"/>
<meta content="랭킹 : 네이버 영화" property="og:title"/>
<meta content="영화, 영화인, 예매, 박스오피스 랭킹 정보 제공" property="og:description"/>
<meta content="article" property="og:type"/>
<meta content="https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&amp;date=20221127" property="og:url"/>
<meta content="http://static.naver.net/m/movie/icons/OG_270_270.png" property="og:image"/><!-- http://static.naver.net/m/movie/im/navermovie.jpg -->
<meta content="http://imgmovie.naver.com/today/naverme/naverme_profile.jpg" property="og:article:thumbnailUrl"/>
<meta content="네이버 영화" property="og:article:author"/

In [170]:
print('영화 제목만 스크래핑')
soup.find('div' , {'class' : 'tit5'})

movie_title_lst = []

for i in range(len(soup.find_all('div' , {'class' : 'tit5'}))) :
    movie_title_lst.append(soup.find_all('div' , {'class' : 'tit5'})[i].a.get_text())
movie_title_lst


영화 제목만 스크래핑


['탑건: 매버릭',
 '올빼미',
 '씽2게더',
 '시네마 천국',
 '극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교',
 '바람과 함께 사라지다',
 '극장판 주술회전 0',
 '벤허',
 '인생은 아름다워',
 '코다',
 '증인',
 '십계',
 '비긴 어게인',
 '타오르는 여인의 초상',
 '윤희에게',
 '매드맥스: 분노의 도로',
 '너의 이름은.',
 '아멜리에',
 '찬실이는 복도 많지',
 '벌새',
 '헤어질 결심',
 '라라랜드',
 '육사오(6/45)',
 '아무르',
 '그녀',
 '자백',
 '공조2: 인터내셔날',
 '폴: 600미터',
 '너의 췌장을 먹고 싶어',
 '러스트 앤 본',
 '초속5센티미터',
 '다가오는 것들',
 '그래비티',
 '에브리씽 에브리웨어 올 앳 원스',
 '언어의 정원',
 '리멤버',
 '데시벨',
 '날씨의 아이',
 '라붐',
 '사랑할 땐 누구나 최악이 된다',
 '블랙 아담',
 '썸머 필름을 타고!',
 '홀리 모터스',
 '아네트',
 '오펀: 천사의 탄생',
 '블랙 팬서: 와칸다 포에버',
 '대무가',
 '동감',
 '120BPM']

In [172]:
print('영화 평점만 스크래핑')
soup.find('td' , {'class' : 'point'}).get_text()

movie_point_lst = []

for i in range(len(soup.find_all('td' , {'class' : 'point'}))) :
    movie_point_lst.append(soup.find_all('td' , {'class' : 'point'})[i].get_text())

movie_point_lst

영화 평점만 스크래핑


['9.76',
 '9.42',
 '9.38',
 '9.33',
 '9.33',
 '9.25',
 '9.24',
 '9.23',
 '9.23',
 '9.13',
 '9.10',
 '9.04',
 '9.04',
 '9.04',
 '8.95',
 '8.81',
 '8.81',
 '8.80',
 '8.77',
 '8.74',
 '8.68',
 '8.62',
 '8.58',
 '8.52',
 '8.51',
 '8.51',
 '8.50',
 '8.43',
 '8.38',
 '8.38',
 '8.36',
 '8.34',
 '8.31',
 '8.31',
 '8.18',
 '8.16',
 '7.99',
 '7.97',
 '7.90',
 '7.74',
 '7.57',
 '7.56',
 '7.48',
 '7.23',
 '6.46',
 '6.31',
 '6.26',
 '6.17',
 '2.77']

In [174]:
movie_frm = pd.DataFrame({
    'title' : movie_title_lst ,
    'point' : movie_point_lst
})
movie_frm

,title,point
0,탑건: 매버릭,9.76
1,올빼미,9.42
2,씽2게더,9.38
3,시네마 천국,9.33
4,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,9.33
5,바람과 함께 사라지다,9.25
6,극장판 주술회전 0,9.24
7,벤허,9.23
8,인생은 아름다워,9.23
9,코다,9.13


In [182]:
movie_frm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   49 non-null     object 
 1   point   49 non-null     float64
dtypes: float64(1), object(1)
memory usage: 912.0+ bytes


In [181]:
movie_frm['point'] = movie_frm['point'].astype(float)

#### 실습

In [183]:
# https://www.imdb.com/title/tt4154756/reviews?ref_=tt_ql_3
# 점수(별점) , 리뷰 제목 , 작성자 , 작성 날짜 , 리뷰 내용
# 데이터 프레임 작성

In [185]:
try :
    html = urlopen('https://www.imdb.com/title/tt4154756/reviews?ref_=tt_ql_3')
except HTTPError as he :
    print('http error')
except URLError  as ue :
    print('url error')
else :
    soup = BeautifulSoup(html.read() , 'html.parser')

In [191]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>어벤져스: 인피니티 워 (2018) - 어벤져스: 인피니티 워 (2018) - User Reviews - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/title/tt4154756/reviews" rel="canonical"/>
<meta content="http://www.imdb.com/title/tt4154756/reviews" property="og:ur

In [444]:
# 점수가 없는 경우가 있다.

rating_lst = []

for i in range(len(soup.find_all('div' ,{ 'class' : 'lister-item-content'}))) :
    if soup.find_all('div' ,{ 'class' : 'lister-item-content'})[i].find('span' , {'class' : 'rating-other-user-rating'}) == None  :
        rating_lst.append(None)
    else :
        rating_lst.append(soup.find_all('div' ,{ 'class' : 'lister-item-content'})[i].find('span' , {'class' : 'rating-other-user-rating'}).span.get_text())
rating_lst

['10',
 None,
 '10',
 '9',
 '10',
 None,
 '10',
 '8',
 '10',
 '8',
 '10',
 '10',
 '10',
 '10',
 '10',
 '10',
 '9',
 '8',
 '10',
 '10',
 '9',
 '9',
 '10',
 '9',
 '10']

In [406]:
date_lst = []

for i in range(len(soup.find_all('span' , {'class' : 'review-date'}))):
    date_lst.append(soup.find_all('span' , {'class' : 'review-date'})[i].get_text())
date_lst
len(date_lst)

25

In [457]:
title_lst = []
review_lst = []
user_name_lst = []
date_lst = []
rating_lst = []

for i in range(len(soup.find_all('div' , {'class' : 'review-container'}))) :
    # title
    title_lst.append(soup.find_all('div' , {'class' : 'review-container'})[i].a.get_text()[1:].replace('\n' , ''))
    # review
    review_lst.append(soup.find_all('div' , {'class' : 'review-container'})[i].find('div' , {'class' : 'text show-more__control'}).get_text())
    # user_name
    user_name_lst.append(soup.find_all('span' , {'class' : 'display-name-link'})[i].get_text())
    # date
    date_lst.append(soup.find_all('span' , {'class' : 'review-date'})[i].get_text())
    # point
    if soup.find_all('div' ,{ 'class' : 'lister-item-content'})[i].find('span' , {'class' : 'rating-other-user-rating'}) == None  :
        rating_lst.append(None)
    else :
        rating_lst.append(soup.find_all('div' ,{ 'class' : 'lister-item-content'})[i].find('span' , {'class' : 'rating-other-user-rating'}).span.get_text())


title_lst
review_lst
user_name_lst
date_lst
rating_lst

review_frm = pd.DataFrame({'리뷰 제목'     : title_lst ,
                           '리뷰 내용'     : review_lst ,
                           '작성자'        : user_name_lst ,
                           '작성 날짜'     : date_lst ,
                           '점수(별점)'    : rating_lst})
review_frm

,리뷰 제목,리뷰 내용,작성자,작성 날짜,점수(별점)
0,"Best movie of the MCU, incredible from start t...",Infinity War is remembered mostly for how it e...,RJBrez,24 January 2021,10
1,Great,This movie portrays a villain that by far has ...,MR-ODIN,23 February 2020,None
2,Best Cliffhanger of all Time,A meticulously planned film down to the last d...,Alex_Lo,5 February 2021,10
3,EMOTIONAL ROLLER COASTER,Avengers infinity war is an emotional roller c...,andreascaloni,16 February 2021,9
4,Better than Endgame,Infinity war is one of the best MCU protects. ...,grztxks,14 March 2021,10
5,A film that pulled off the impossible.,Avengers: Infinity War is a film that should b...,cartermain,20 January 2021,None
6,EPIC,Infinity war seemed much better than endgame t...,lalit28595,27 July 2021,10
7,A superior avengers sequel,Although this film has 'Avengers' in the title...,Abdulxoxo,5 February 2021,8
8,How MCU should have always been like,It was only after the release of Infinity War ...,Puroo,21 April 2021,10
9,It all led to this: superhero film at its best,#MCUrewatch. A confrontation that has been in ...,Filmdokter,25 October 2021,8


In [440]:
review_frm

,title,review,user_name,date,rating
0,"Best movie of the MCU, incredible from start t...",Infinity War is remembered mostly for how it e...,RJBrez,24 January 2021,10
1,Great,This movie portrays a villain that by far has ...,MR-ODIN,23 February 2020,None
2,Best Cliffhanger of all Time,A meticulously planned film down to the last d...,Alex_Lo,5 February 2021,10
3,EMOTIONAL ROLLER COASTER,Avengers infinity war is an emotional roller c...,andreascaloni,16 February 2021,9
4,Better than Endgame,Infinity war is one of the best MCU protects. ...,grztxks,14 March 2021,10
5,A film that pulled off the impossible.,Avengers: Infinity War is a film that should b...,cartermain,20 January 2021,None
6,EPIC,Infinity war seemed much better than endgame t...,lalit28595,27 July 2021,10
7,A superior avengers sequel,Although this film has 'Avengers' in the title...,Abdulxoxo,5 February 2021,8
8,How MCU should have always been like,It was only after the release of Infinity War ...,Puroo,21 April 2021,10
9,It all led to this: superhero film at its best,#MCUrewatch. A confrontation that has been in ...,Filmdokter,25 October 2021,8


#### 페이지 처리방법 익히기

In [ ]:
# 실습 2
https://search.naver.com/search.naver?query=python&sm=tab_pge&where=nexearch
https://search.naver.com/search.naver?display=15&f=&filetype=0&page=2&query=python&research_url=&sm=tab_pge&start=1&where=web    

In [459]:
from time import sleep, time
from random import randint 
from IPython.core.display import clear_output

In [513]:
start_time = time()
requests = 0
for idx in range(5):
    requests += 1
    print('request - ' , requests)
    
    # 시간 생성(초)
    sleep(randint(1,5))
    
    current_time = time()
    elapsed_time = current_time - start_time
    
    print('request '  , requests , 'elapsed_time' , requests/elapsed_time)
    # 마지막만 출력
    clear_output(wait = True)
    

request -  5
request  5 elapsed_time 0.4149756119076348


In [512]:
time()

1669612858.9243498

In [610]:
pages = [str(i) for i in range(1,5)]
pages

['1', '2', '3', '4']

In [611]:
soup_lst = []

for page in pages : 
    try :
        html = urlopen('https://ad.search.naver.com/search.naver?where=ad&sm=svc_nrs&query=python&referenceId=hGeaYsprvxZsshy%20j5Cssssst9h-276470&bucketTest=impression-neo-97&bucket=0&pagingIndex='+page)
    except HTTPError as he :
        print('http error')
    except URLError  as ue :
        print('url error')
    else :
        soup = BeautifulSoup(html.read() , 'html.parser')
        soup_lst.append(soup)

In [676]:
for page in range(len(soup_lst)) :
    globals()['Page_0{}'.format(page+1)] = []    
    for i in range(len(soup_lst[page].find_all('a' , {'class' : 'tit_wrap'}))) :
        globals()['Page_0{}'.format(page+1)].append(soup_lst[page].find_all('a' , {'class' : 'tit_wrap'})[i].find('span' , {'class' : 'lnk_tit'}).get_text())
        
# for page in range(1, len(soup_lst)+1) :
#     print(globals()['Page_0{}'.format(page)])


AD_frm = pd.DataFrame([ globals()['Page_0{}'.format(page)] for page in range(1 , len(soup_lst)+1)]).T
AD_frm.columns = ['page_0'+str(num+1) for num in list(AD_frm.columns)]
display(AD_frm)

,page_01,page_02,page_03,page_04
0,개발자 채용 공고 원티드,11번가 PYTHON,"LH 강의, 클래스유",코리아IT아카데미 강선우
1,그린아이티아카데미학원,파이썬 강의는 인프런에서!,"인공지능 블럭코딩, 파이션",아이티뱅크 신규상담 김지호
2,그린컴퓨터아트학원공식홈페이지,KGITBANK학원 종로본원,MIT 테크 리뷰 한국어판,와이즈코딩러닝센터안양점
3,SBS아카데미프로그래밍학원,온빛정보올림피아드 코딩 교육,PYTHON 컴알,코리아IT아카데미 조재우
4,"PYTHON 전문가, 크몽",통계 SW SPSS 교육,코딩 IT 초급반학원 홍기민,두런 IT클래스 추천
5,메가스터디 IT PYTHON,PYTHON 모든에듀,에이콘아카데미 강남캠퍼스,KG아이티뱅크 전혜원 멘토
6,그린컴퓨터아카데미 강남캠퍼스,IT뱅크 취업지원실 이유진,신세계몰 PYTHON,돌고넷
7,스파르타코딩클럽 파이썬 강의,KG아이티뱅크 이유진 선생님,ITBANK 파이썬 이원우,AI/블록체인 스마트금융과
8,에듀윌 국비교육원,IT 전공대비 교육 신율기,쿠팡 PYTHON,코리아 IT아카데미 조수정
9,더조은컴퓨터아카데미 공식,IT교육전문 IT뱅크 복영일,AI 인공지능 카톡분석,등촌역 PYTHON학원


In [709]:
text_lst = []
for cnt in soup_lst[0].find_all('a' , {'class' : 'tit_wrap'})[0].find_all('span' , {'class' : 'lnk_tit'}) :
    text_lst.append(cnt.get_text())
text_lst

one_line(text_lst)

['개발자 채용 공고 원티드', '실시간 직군별 연봉 정보', '합격률 4배 포지션 제안']

In [724]:
def one_line( lst : list) :
    string = ''
    for i in lst :
        string = string + i + '*'
    return string

text_lst = []

for page in range(len(soup_lst)) :
    globals()['Page_0{}'.format(page+1)] = []    
    for i in range(len(soup_lst[page].find_all('a' , {'class' : 'tit_wrap'}))) :
        for cnt in soup_lst[page].find_all('a' , {'class' : 'tit_wrap'})[i].find_all('span' , {'class' : 'lnk_tit'}) :
            text_lst.append(cnt.get_text())
        globals()['Page_0{}'.format(page+1)].append(one_line(text_lst))
        text_lst = []
        
        
# for page in range(1, len(soup_lst)+1) :
#     print(globals()['Page_0{}'.format(page)])


AD_frm = pd.DataFrame([ globals()['Page_0{}'.format(page)] for page in range(1 , len(soup_lst)+1)]).T
AD_frm.columns = ['page_0'+str(num+1) for num in list(AD_frm.columns)]
display(AD_frm)

,page_01,page_02,page_03,page_04
0,개발자 채용 공고 원티드*실시간 직군별 연봉 정보*합격률 4배 포지션 제안*,11번가 PYTHON*실시간 초특가 확인하기*,"LH 강의, 클래스유*",코리아IT아카데미 강선우*선착순 특가 이벤트 OPEN*
1,그린아이티아카데미학원*국비최대100%무료교육*IT 및 SW개발 전문교육*,파이썬 강의는 인프런에서!*회원전용 성장응원쿠폰 드려요*,"인공지능 블럭코딩, 파이션 *인공지능 코딩교육*활용교재 판매, 교육영상*",아이티뱅크 신규상담 김지호*
2,그린컴퓨터아트학원공식홈페이지*고용노동부지정 우수훈련기관*,KGITBANK학원 종로본원*신규입학상담 추가할인*,MIT 테크 리뷰 한국어판*,와이즈코딩러닝센터안양점*코딩앤플레이 안양.평촌지사*
3,SBS아카데미프로그래밍학원*국비 100% 무료교육 상담*,온빛정보올림피아드 코딩 교육*,PYTHON 컴알*따라하다보면 나도 전문가*,코리아IT아카데미 조재우*
4,"PYTHON 전문가, 크몽*No.1 프리랜서 마켓 크몽*신규회원 10만원 쿠폰팩*",통계 SW SPSS 교육*통계 교육 전문*데이터 분석 교육 전문*,코딩 IT 초급반학원 홍기민*,두런 IT클래스 추천*모든 지식은 두런에서!*비용 없는 무료 플랫폼*
5,"메가스터디 IT PYTHON*데이터, 마케팅 필수 언어!*겨울방학 스페셜 이벤트!*",PYTHON 모든에듀*온라인강의 추천*,"에이콘아카데미 강남캠퍼스*빅데이터교육 R+Python*AI(머신러닝,딥러닝)*",KG아이티뱅크 전혜원 멘토*단기과정 취업보장 부캠*
6,그린컴퓨터아카데미 강남캠퍼스*22년사회공헌대상교육부장관상*,IT뱅크 취업지원실 이유진*첫 문의 남기면 인강무료제공*,신세계몰 PYTHON*짜릿한 특가!*,돌고넷*
7,스파르타코딩클럽 파이썬 강의*5주만에 왕초보 탈출*완주까지 찐~한 학습 관리*,KG아이티뱅크 이유진 선생님*프로그래밍/코딩/서버/보안*,ITBANK 파이썬 이원우*IT우수교육기관 무료교육상담*,AI/블록체인 스마트금융과*
8,에듀윌 국비교육원*1:1 맞춤 취업 컨설팅*,IT 전공대비 교육 신율기*전공대비 4주완성 교육*,쿠팡 PYTHON*로켓와우멤버 무제한 무료배송*,코리아 IT아카데미 조수정*
9,더조은컴퓨터아카데미 공식*고용노동부 장관상 수상*고용노동부 지정 5년우수기관*,IT교육전문 IT뱅크 복영일*1:1 전문가 무료상담*,AI 인공지능 카톡분석*,등촌역 PYTHON학원*


In [725]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

url_base = 'https://www.koreabaseball.com/Record/Player/{category}'
category_list = ['HitterBasic/Basic1.aspx', 'PitcherBasic/Basic1.aspx', 
                 'Defense/Basic.aspx', 'Runner/Basic.aspx']

data_list = []

for category in category_list:
    url = url_base.format(category = category)
    req = requests.get(url)

    html = req.text
    soup = bs(html, 'html.parser')
    
    temp_table = soup.find('div', {'class':'record_result'})
    #### get table info
    
    col_tag = temp_table.find_all('td')
    col_list = ['Rank', 'Name', 'Team']
    for col in col_tag:
        try:
            temp_value = col.attrs['data-id']
            if temp_value not in col_list:
                col_list.append(temp_value)
            else:
                break
        except:
            pass
        
    temp_data = pd.DataFrame(columns = col_list)

        
    i = 0
    index = 0
    col_len = len(col_list)
    while True:
        try:
            temp_data.loc[i] = [x.text for x in temp_table.find_all('td')[index:index + col_len]]
            i += 1
            index += col_len
        except:
            break
    
    data_list.append(temp_data)

In [726]:
data_list

[Empty DataFrame
 Columns: [Rank, Name, Team]
 Index: [],
 Empty DataFrame
 Columns: [Rank, Name, Team]
 Index: [],
    Rank   Name Team POS_SC GAME_CN START_GAME_CN DEFEN_INN2_CN ERR_CN POFF_CN  \
 0     1    배정대   KT    중견수     144           134      1195 2/3      3       0   
 1     1    박해민   LG    중견수     144           142          1205      2       0   
 2     3    오지환   LG    유격수     142           137          1167     16       0   
 3     4    황재균   KT    3루수     138           133      1147 2/3     12       0   
 4     4    송성문   키움    3루수     138           137          1216     15       0   
 5     4    박성한  SSG    유격수     138           131          1176     24       0   
 6     7    이지영   키움     포수     137           102       994 2/3     11       0   
 7     7    김선빈  KIA    2루수     137           134      1134 1/3     10       0   
 8     9    유강남   LG     포수     134           115      1008 1/3      7       1   
 9     9    이정후   키움    중견수     134           132      1155 1/3 

In [727]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

url_base = 'https://www.koreabaseball.com/Record/Player/{category}'
category_list = ['HitterBasic/Basic1.aspx', 'PitcherBasic/Basic1.aspx', 
                 'Defense/Basic.aspx', 'Runner/Basic.aspx']

data_list = []

for category in category_list:
    url = url_base.format(category = category)
    req = requests.get(url)

    html = req.text
    soup = bs(html, 'html.parser')
    
    print(soup.find('div', {'class':'record_result'}))
    #### get table info
    
    

<div class="record_result">
<table cellpadding="0" cellspacing="0" class="tData01 tt" summary="선수 기본기록으로 경기,타석,타수,득점,안타,2루타, 3루타 등을 표시합니다">
<colgroup>
<col width="50"/>
<col width="80"/>
<col width="50"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="70"/>
<col width="60"/>
<col width="60"/>
</colgroup>
<thead>
<tr>
<th>순위</th>
<th>선수명</th>
<th>팀명</th>
<th><a href="javascript:sort('HRA_RT');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="타율">AVG</a></th>
<th><a href="javascript:sort('GAME_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="경기">G</a></th>
<th><a href="javascript:sort('PA_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="타석">PA</a></th>
<th><a href="javascript:sort('AB_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="타수">AB</a></th>
<th><a href="javascript:sort('RUN_CN

<div class="record_result">
<table cellpadding="0" cellspacing="0" class="tData01 tt" summary="선수 수비기록으로 포지션,경기, 선발,실책,견제사, 자살,보살,병살,수비율,포일,도루허용, 도루저지, 도루저지율을 나타냅니다">
<colgroup>
<col width="50"/>
<col width="80"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="80"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
<col width="50"/>
</colgroup>
<thead>
<tr>
<th>순위</th>
<th>선수명</th>
<th>팀명</th>
<th><a href="javascript:sort('POS_SC');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="포지션">POS</a></th>
<th><a href="javascript:sort('GAME_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="경기">G</a></th>
<th><a href="javascript:sort('START_GAME_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="선발경기">GS</a></th>
<th><a href="javascript:sort('DEFEN_INN2_CN');" onmouseenter="tooltip(this)" onmouseout="hideTi

<div class="record_result">
<table cellpadding="0" cellspacing="0" class="tData01 tt" summary="선수 주루기록으로 순위,선수명,팀명,경기,도루시도,도루성공,도루실패,도루성공률,주루사,견제사를 표시합니다">
<colgroup>
<col width="80"/>
<col width="120"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
<col width="80"/>
</colgroup>
<thead>
<tr>
<th>순위</th>
<th>선수명</th>
<th>팀명</th>
<th><a href="javascript:sort('GAME_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="경기">G</a></th>
<th><a href="javascript:sort('SBA_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루시도">SBA</a></th>
<th><a href="javascript:sort('SB_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루허용">SB</a></th>
<th><a href="javascript:sort('CS_CN');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루실패">CS</a></th>
<th><a href="javascript:sort('SB_RT');" onmouseenter="tooltip(this)" onmouseout="hideTip()" title="도루성공률">SB%</a></th>
<t